In [1]:
from docksuitex import Protein, Ligand, PocketFinder, VinaDocking, AD4Docking
from docksuitex.utils import fetch_pdb, fetch_sdf, parse_vina_log_to_csv, parse_ad4_dlg_to_csv

In [ ]:
Protein_path = fetch_pdb(pdbid = "1HVR")

In [ ]:
prot = Protein(input= Protein_path, fix_pdb =True, remove_heterogens=True, add_hydrogens=True, remove_water=True, add_charges=True, preserve_charge_types=None)
prot.prepare() #Default parameters
prot.view_molecule()

In [4]:
# Find pockets using p2rank
pf = PocketFinder(input="1HVR.pdbqt", cpu=6)
pockets = pf.run()

✅ Pocket prediction completed. Found 2 pockets.
Pocket 1 center: (-9.2421, 16.1006, 27.6026)
Pocket 2 center: (-12.5447, 21.7728, 45.5697)


In [ ]:
Ligand_path = fetch_sdf(cid = "338")

In [ ]:
# Fetch and prepare ligand

lig = Ligand(input=Ligand_path, minimize=None,  remove_water=True, add_hydrogens=True, add_charges=True, preserve_charge_types=None)
lig.prepare() #Default parameters

lig.view_molecule()

## Perform protein ligand docking at the first pocket

#### Using AutoDock Vina


In [ ]:
center = pockets[0]['center']

vina = VinaDocking(
    receptor="1HVR.pdbqt",
    ligand="338.pdbqt",
    grid_center=center,
    
    # Default Parameters
    grid_size=(20,20,20),
    exhaustiveness=8,
    num_modes=9,
    cpu=11
)
vina.run(save_to="vina_results/1HVR_338")
vina.view_results()

# Parse Vina results
parse_vina_log_to_csv(results_dir="vina_results", output_csv="vina_results/vina_summary.csv")


#### Using AutoDock4

In [ ]:
# Perform docking with the first pocket
# Using AutoDock4
ad4 = AD4Docking(
    receptor="1HVR.pdbqt",
    ligand="338.pdbqt",
    grid_center=pockets[0]['center'],

    #Default Parameters
    grid_size=(60,60,60),
    spacing=0.375,     
    dielectric=-0.1465, 
    smooth=0.5,          
    ga_pop_size=150,
    ga_num_evals=2500000,
    ga_num_generations=27000,
    ga_elitism=1,
    ga_mutation_rate=0.02,
    ga_crossover_rate=0.8,
    ga_run=10,
    rmstol=2.0,
)

ad4.run(save_to="ad4_results/1HVR_338")    
ad4.view_results()

# Parse AD4 results
parse_ad4_dlg_to_csv(results_dir="ad4_results", output_csv="ad4_results/ad4_summary.csv")